In [1]:
f= read("D:/federalistpapers.txt", String)

subEssay = String[]

i = 1
while i != nothing 
    next_occurrence_start = findnext("FEDERALIST No.", f, i)
    next_occurrence_end = findnext("PUBLIUS", f, i)
    println(SubString(f, next_occurrence_start[1], next_occurrence_end[2]))
    if next_occurrence_start == nothing
        break
    end
    i = next_occurrence_end[1] + 1
end


FEDERALIST No. 1

General Introduction

For the Independent Journal. Saturday, October 27, 1787


HAMILTON

To the People of the State of New York:

AFTER an unequivocal experience of the inefficacy of the subsisting
federal government, you are called upon to deliberate on a new
Constitution for the United States of America. The subject speaks its
own importance; comprehending in its consequences nothing less than the
existence of the UNION, the safety and welfare of the parts of which it
is composed, the fate of an empire in many respects the most interesting
in the world. It has been frequently remarked that it seems to have been
reserved to the people of this country, by their conduct and example,
to decide the important question, whether societies of men are really
capable or not of establishing good government from reflection and
choice, or whether they are forever destined to depend for their
political constitutions on accident and force. If there be any truth
in the remark, the 


FEDERALIST No. 3

The Same Subject Continued (Concerning Dangers From Foreign Force and
Influence)

For the Independent Journal. Saturday, November 3, 1787

JAY

To the People of the State of New York:

IT IS not a new observation that the people of any country (if, like
the Americans, intelligent and wellinformed) seldom adopt and steadily
persevere for many years in an erroneous opinion respecting their
interests. That consideration naturally tends to create great respect
for the high opinion which the people of America have so long and
uniformly entertained of the importance of their continuing firmly
united under one federal government, vested with sufficient powers for
all general and national purposes.

The more attentively I consider and investigate the reasons which appear
to have given birth to this opinion, the more I become convinced that
they are cogent and conclusive.

Among the many objects to which a wise and free people find it necessary
to direct their attention, that

FEDERALIST No. 5

The Same Subject Continued (Concerning Dangers From Foreign Force and
Influence)

For the Independent Journal. Saturday, November 10, 1787

JAY

To the People of the State of New York:

QUEEN ANNE, in her letter of the 1st July, 1706, to the Scotch
Parliament, makes some observations on the importance of the UNION then
forming between England and Scotland, which merit our attention. I shall
present the public with one or two extracts from it: "An entire and
perfect union will be the solid foundation of lasting peace: It will
secure your religion, liberty, and property; remove the animosities
amongst yourselves, and the jealousies and differences betwixt our two
kingdoms. It must increase your strength, riches, and trade; and by
this union the whole island, being joined in affection and free from all
apprehensions of different interest, will be ENABLED TO RESIST ALL ITS
ENEMIES." "We most earnestly recommend to you calmness and unanimity in
this great and weighty affai


FEDERALIST No. 7

The Same Subject Continued (Concerning Dangers from Dissensions Between
the States)

For the Independent Journal. Thursday, November 15, 1787

HAMILTON

To the People of the State of New York:

IT IS sometimes asked, with an air of seeming triumph, what inducements
could the States have, if disunited, to make war upon each other? It
would be a full answer to this question to say--precisely the same
inducements which have, at different times, deluged in blood all the
nations in the world. But, unfortunately for us, the question admits
of a more particular answer. There are causes of differences within
our immediate contemplation, of the tendency of which, even under the
restraints of a federal constitution, we have had sufficient experience
to enable us to form a judgment of what might be expected if those
restraints were removed.

Territorial disputes have at all times been found one of the most
fertile sources of hostility among nations. Perhaps the greatest
proport

FEDERALIST No. 8

The Consequences of Hostilities Between the States

From the New York Packet. Tuesday, November 20, 1787.

HAMILTON

To the People of the State of New York:

ASSUMING it therefore as an established truth that the several States,
in case of disunion, or such combinations of them as might happen to be
formed out of the wreck of the general Confederacy, would be subject to
those vicissitudes of peace and war, of friendship and enmity, with
each other, which have fallen to the lot of all neighboring nations not
united under one government, let us enter into a concise detail of some
of the consequences that would attend such a situation.

War between the States, in the first period of their separate existence,
would be accompanied with much greater distresses than it commonly is
in those countries where regular military establishments have long
obtained. The disciplined armies always kept on foot on the continent
of Europe, though they bear a malignant aspect to liberty an

...

FEDERALIST No. 37

Concerning the Difficulties of the Convention in Devising a Proper Form
of Government.

From the Daily Advertiser. Friday, January 11, 1788.

MADISON

To the People of the State of New York:

IN REVIEWING the defects of the existing Confederation, and showing that
they cannot be supplied by a government of less energy than that before
the public, several of the most important principles of the latter
fell of course under consideration. But as the ultimate object of
these papers is to determine clearly and fully the merits of this
Constitution, and the expediency of adopting it, our plan cannot be
complete without taking a more critical and thorough survey of the work
of the convention, without examining it on all its sides, comparing
it in all its parts, and calculating its probable effects. That this
remaining task may be executed under impressions conducive to a just
and fair result, some reflections must in this place be indulged, which
candor previously suggests.


FEDERALIST No. 39

The Conformity of the Plan to Republican Principles

For the Independent Journal. Wednesday, January 16, 1788

MADISON

To the People of the State of New York:

THE last paper having concluded the observations which were meant to
introduce a candid survey of the plan of government reported by
the convention, we now proceed to the execution of that part of our
undertaking.

The first question that offers itself is, whether the general form and
aspect of the government be strictly republican. It is evident that
no other form would be reconcilable with the genius of the people of
America; with the fundamental principles of the Revolution; or with that
honorable determination which animates every votary of freedom, to
rest all our political experiments on the capacity of mankind for
self-government. If the plan of the convention, therefore, be found to
depart from the republican character, its advocates must abandon it as
no longer defensible.

What, then, are the disti


FEDERALIST No. 40

On the Powers of the Convention to Form a Mixed Government Examined and
Sustained For the New York Packet. Friday, January 18, 1788.

MADISON

To the People of the State of New York:

THE SECOND point to be examined is, whether the convention were
authorized to frame and propose this mixed Constitution.

The powers of the convention ought, in strictness, to be determined
by an inspection of the commissions given to the members by their
respective constituents. As all of these, however, had reference, either
to the recommendation from the meeting at Annapolis, in September, 1786,
or to that from Congress, in February, 1787, it will be sufficient to
recur to these particular acts.

The act from Annapolis recommends the "appointment of commissioners to
take into consideration the situation of the United States; to devise
SUCH FURTHER PROVISIONS as shall appear to them necessary to render the
Constitution of the federal government ADEQUATE TO THE EXIGENCIES OF THE
UNION

Excessive output truncated after 538254 bytes.

MethodError: MethodError: no method matching getindex(::Nothing, ::Int64)

In [ ]:
text = read("C:/Users/ADMIN/OneDrive/Documents/GitHub/csds133-s23-2/federalistpapers.txt", String)
essays = split(text, "Federalist No.")
println(essays)